# 📊 Análisis de Ofertas Laborales en Ecuador (Completo)
Este notebook carga, limpia y analiza un dataset de ofertas de trabajo en Ecuador.
Incluye tablas, gráficos (Seaborn/Matplotlib) y un mini dashboard con Plotly.

In [ ]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import unicodedata

sns.set(style="whitegrid")


## 1. Cargar datos

In [ ]:

from google.colab import files

# Subir archivo limpio exportado previamente
uploaded = files.upload()

df = pd.read_csv("ofertas_ec_limpio.csv")
df.head()


## 2. Filtros simples

In [ ]:

# Puedes cambiar estos valores para filtrar la información
ciudad_filtro = "quito"
area_filtro = None
nivel_filtro = None

df_filtrado = df.copy()
if ciudad_filtro:
    df_filtrado = df_filtrado[df_filtrado['ciudad_norm'] == ciudad_filtro]
if area_filtro:
    df_filtrado = df_filtrado[df_filtrado['areaEstudios'] == area_filtro]
if nivel_filtro:
    df_filtrado = df_filtrado[df_filtrado['nivelInstruccion'] == nivel_filtro]

df_filtrado.head()


## 3. Ranking de cargos más frecuentes

In [ ]:

top_cargos = df['cargo'].value_counts().head(20)
plt.figure(figsize=(8,6))
sns.barplot(y=top_cargos.index, x=top_cargos.values, palette="viridis")
plt.title("Top 20 Cargos más frecuentes")
plt.xlabel("Número de Ofertas")
plt.ylabel("Cargo")
plt.show()


## 4. Heatmap Ciudad × Área de Estudio

In [ ]:

tabla_heatmap = pd.crosstab(df['ciudad_norm'], df['areaEstudios'])
plt.figure(figsize=(12,8))
sns.heatmap(tabla_heatmap, cmap="YlGnBu", cbar_kws={'label': 'Número de ofertas'})
plt.title("Heatmap Ciudad × Área de Estudio")
plt.show()


## 5. Estacionalidad: Ofertas por mes

In [ ]:

ofertas_mes = df.groupby('ym_publicacion').size().reset_index(name="num_ofertas")
plt.figure(figsize=(12,5))
sns.lineplot(x="ym_publicacion", y="num_ofertas", data=ofertas_mes, marker="o")
plt.xticks(rotation=45)
plt.title("Evolución mensual de ofertas")
plt.show()


## 6. Detección de outliers en remuneración

In [ ]:

plt.figure(figsize=(10,5))
sns.boxplot(x=df['remuneracion_mid'])
plt.title("Boxplot de remuneración media (USD/mes)")
plt.show()

# Calcular outliers usando regla de IQR
Q1 = df['remuneracion_mid'].quantile(0.25)
Q3 = df['remuneracion_mid'].quantile(0.75)
IQR = Q3 - Q1
outliers = df[(df['remuneracion_mid'] < Q1 - 1.5*IQR) | (df['remuneracion_mid'] > Q3 + 1.5*IQR)]
print("Número de outliers:", len(outliers))
outliers[['cargo','ciudad_titulo','remuneracion_mid']].head()


## 7. Dashboard simple con Plotly

In [ ]:

fig1 = px.bar(top_cargos.reset_index().rename(columns={'index':'cargo', 'cargo':'num_ofertas'}),
              x="num_ofertas", y="cargo", orientation="h", title="Top 20 Cargos")

fig2 = px.histogram(df, x="remuneracion_mid", nbins=30, title="Distribución de remuneración")

fig3 = px.line(ofertas_mes, x="ym_publicacion", y="num_ofertas", title="Ofertas por mes")

fig1.show()
fig2.show()
fig3.show()
